In [2]:
import sys
sys.version

'3.7.6 (default, Jan  8 2020, 19:59:22) \n[GCC 7.3.0]'

# import

In [3]:
import pandas as pd

In [3]:
train = pd.read_csv('../data/w_review_train.csv',header=None)
train_ori = pd.read_csv('../data/w_review_train.csv',header=None)
train.columns = ['review']
train_ori.columns = ['review']
print(f"train shape is : {train.shape}")

train shape is : (40000, 1)


# Preprocess

In [4]:
import pythainlp
from pythainlp.tokenize import word_tokenize
import re

In [5]:
def clean_special_char(train):

    special_char = r"[\\\@\^\%\(\*\|\<\`\.\_\=\!\>\;\?\#\$\)\/\,\ๆ\'\~\:\+\-\"\฿]"
    space_char = r"[\n\&]"
    num = '[0-9]'
    num_th = '[๑๒๓๓๔๕๖๗๘๙]'
    eng_words = '[A-Za-z]'
    
    train['review'] = train['review'].str.replace(special_char, ' ')
    train['review'] = train['review'].str.replace(space_char, ' ')
    train['review'] = train['review'].str.replace(num, ' ', regex=True)
    train['review'] = train['review'].str.replace(num_th, ' ', regex=True)
    train['review'] = train['review'].str.replace(eng_words, ' ', regex=True)
    train['review'] = train['review'].str.lower()
    
    return train

Tokenization

In [16]:
%%time
train = clean_special_char(train)
train['tokenized'] = train['review'].apply(lambda x: word_tokenize(x,engine='attacut',keep_whitespace=False, custom_dict=trie))
train.to_pickle('tokenized_with_wongnai_customdict.pkl')

CPU times: user 2h 11min 6s, sys: 2min 25s, total: 2h 13min 32s
Wall time: 16min 45s


In [5]:
def load_dict(file_nm):
    bigram_custom = pd.read_csv(file_nm,header=None,usecols=[0])
    bigram_custom = bigram_custom[0].tolist()
    return bigram_custom

# top words dict
bigram_dict = load_dict('../dict/bigram_customdict_top100.txt')
trigram_dict = load_dict('../dict/trigram_customdict_top1000.txt')
quadgram_dict = load_dict('../dict/quadgram_customdict_top300.txt')
top_words_customdict = set(bigram_dict + trigram_dict + quadgram_dict)

# food dict
custom_dict_wongnai = pd.read_csv('../data/food_dictionary.txt',header=None,names=['word'])
food_dict = custom_dict_wongnai['word'].tolist()
custom_dict_wongnai = set(food_dict)

# # union
# custom_dict = custom_dict_wongnai.union(top_words_customdict)
# trie = pythainlp.tokenize.dict_trie(dict_source=custom_dict)

# # if use deepcut directly
# tokenize_dict = list(trie.words)
# tokenize_dict = sorted(tokenize_dict, key=len)

In [27]:
# sample sorted list
tokenize_dict[:5], tokenize_dict[-5:]

(['ดีนะ', 'ดีมาก', 'รอคิว', 'มาลอง', 'คอหมู'],
 ['ไก่ต้นตำรับบุญตงกี่ เนื้อน่อง สะโพก',
  'ไก่ย่าง เนื้อปลากระพงทอดผัดเหล้าจีน',
  'ข้าวไข่กระทะหน้ากุ้งกระเทียมพริกไทย',
  'chicken schnitzel with italian slaw',
  'ทอดมันปลาหมึก แหนมเนือง กุ้งพันอ้อย'])

In [6]:
tokenize_dict = food_dict + bigram_dict + trigram_dict + quadgram_dict

In [7]:
# sample sorted list by the priority
tokenize_dict[:5], tokenize_dict[-5:]

(['โรตีทุเรียน',
  'ตำร้อยเอ็ด',
  'สันคอย่างแจ่ว',
  'เกี๊ยวกุ้งทอดครีมสลัด',
  'ส้มตำปูนึ่ง'],
 ['ก็ไม่ผิดหวัง',
  'ที่จอดรถสะดวก',
  'ให้เลือกหลายอย่าง',
  'ถือว่าคุ้มค่า',
  'กระพงทอดน้ำปลา'])

In [10]:
%%time
train = clean_special_char(train)
train['tokenized'] = train['review'].apply(lambda x: word_tokenize(x,engine='new_mm',keep_whitespace=False, custom_dict=trie))
train.to_pickle('newmm_tokenized_with_wongnai_topwords_customdict.pkl')

CPU times: user 1min 2s, sys: 782 ms, total: 1min 3s
Wall time: 1min 3s


## debug : tokenized by custom_dict

In [9]:
import deepcut
debug = clean_special_char(train[:5].copy())
debug

,review
0,ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำ...
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก ร้านที่...
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลาย คนอาจจะสงสั...
4,อร่อย เดินผ่าน ทุกวัน ไม่ยักร...


In [18]:
%%time
debug_res = debug['review'].apply(lambda x: deepcut.tokenize(x, custom_dict=tokenize_dict))

CPU times: user 2min 7s, sys: 235 ms, total: 2min 7s
Wall time: 2min 5s


In [19]:
%%time
debug['review_nospace'] = debug['review'].str.replace(' ', '')
debug_res2 = debug['review_nospace'].apply(lambda x: deepcut.tokenize(x, custom_dict=tokenize_dict))

CPU times: user 2min 6s, sys: 225 ms, total: 2min 7s
Wall time: 2min 5s


it uses the same amount of time for the both library > so, choosing the pythainlp for the sake of flexibility

In [20]:
debug_res[0]

['ร้าน',
 'อาหาร',
 'ใหญ่มาก',
 'กกกกกก',
 ' ',
 ' ',
 'เลี้ยว',
 'เข้า',
 'มา',
 'เจอ',
 'ห้องน้ำ',
 'ก่อน',
 'เลย',
 ' ',
 'เออแปลก',
 'ดี',
 ' ',
 ' ',
 'ห้อง',
 'ทาน',
 'หลัก',
 ' ',
 'อยู่',
 'ชั้น',
 ' ',
 ' ',
 ' ',
 'มี',
 'กาแฟ',
 ' ',
 'น้ำผึ้ง',
 ' ',
 'ซึ่ง',
 'ก็',
 'แค่',
 'เอา',
 'น้ำผึ้ง',
 'มา',
 'ราด',
 ' แพงเวอร์',
 ' ',
 'อย่า',
 'สั่งเลย',
 ' ',
 ' ',
 'ลาบไข่ต้ม',
 ' ',
 'ไข่',
 'มัน',
 'คาว',
 'อะ',
 ' ',
 'เลย',
 'ไม่',
 'ประทับใจ',
 'เท่า',
 'ไหร่',
 ' ',
 'ทอดมันหัวปลี',
 'กรอบ',
 'อร่อย',
 'ต้อง',
 'เบิ้ล',
 ' ',
 ' ',
 'พะแนงห่อไข่',
 'อร่อยดี',
 ' ',
 'เห้ยแต่',
 'ราคา',
 ' ',
 ' ',
 ' ',
 ' ',
 'บาท',
 'มัน',
 'เกิน',
 'ไป',
 'นะ',
 ' ',
 'รับ',
 'ไม่',
 'ไหวว',
 ' ',
 'เลิก',
 'กิน',
 'แล้ว',
 'มี',
 'ขนมหวาน',
 'ให้',
 'กิน',
 'ฟรี',
 'เล็กน้อย',
 ' ',
 ' ',
 'ขนมไทย',
 ' ',
 ' ',
 ' ',
 ' ',
 'คง',
 'ไม่',
 'ไป',
 'ซ้ำ',
 ' ',
 'แพงเกิน',
 ' ',
 ' ',
 ' ']

In [21]:
debug_res2[0]

['ร้าน',
 'อาหาร',
 'ใหญ่มาก',
 'กกกกกก',
 'เลี้ยว',
 'เข้า',
 'มา',
 'เจอ',
 'ห้องน้ำ',
 'ก่อน',
 'เลย',
 'เออ',
 'แปลก',
 'ดี',
 'ห้อง',
 'ทาน',
 'หลัก',
 'อยู่',
 'ชั้น',
 'มี',
 'กาแฟน้ำผึ้ง',
 'ซึ่ง',
 'ก็',
 'แค่',
 'เอา',
 'น้ำผึ้ง',
 'มา',
 'ราด',
 'แพงเวอร์',
 'อย่า',
 'สั่งเลย',
 'ลาบไข่ต้ม',
 'ไข่',
 'มัน',
 'คาว',
 'อะเลย',
 'ไม่',
 'ประทับใจ',
 'เท่า',
 'ไหร่',
 'ทอดมันหัวปลี',
 'กรอบ',
 'อร่อย',
 'ต้อง',
 'เบิ้ล',
 'พะแนงห่อไข่',
 'อร่อยดี',
 'เห้ย',
 'แต่',
 'ราคา',
 'บาท',
 'มัน',
 'เกิน',
 'ไป',
 'นะ',
 'รับ',
 'ไม่',
 'ไหวว',
 'เลิก',
 'กิน',
 'แล้ว',
 'มี',
 'ขนมหวาน',
 'ให้',
 'กิน',
 'ฟรี',
 'เล็กน้อย',
 'ขนมไทย',
 'คง',
 'ไม่',
 'ไป',
 'ซ้ำ',
 'แพง',
 'เกิน']

key take aways here :
1. the deep cut will process the cutting word from the endding of the list (so, placed the importance keyword at last!!!)

In [22]:
debug_trie = pythainlp.tokenize.dict_trie(dict_source=tokenize_dict)

In [23]:
%%time
debug['review_nospace'] = debug['review'].str.replace(' ', '')
debug_res3 = debug['review_nospace'].apply(lambda x: pythainlp.word_tokenize(x,
                                                                             engine='attacut',
                                                                             keep_whitespace=False,
                                                                             custom_dict=debug_trie))

CPU times: user 1.54 s, sys: 137 ms, total: 1.68 s
Wall time: 7.52 s


In [24]:
debug_res3[0]

['ร้าน',
 'อาหาร',
 'ใหญ่',
 'มาก',
 'กกกกกก',
 'เลี้ยว',
 'เข้า',
 'มา',
 'เจอ',
 'ห้องน้ำ',
 'ก่อน',
 'เลย',
 'เออ',
 'แปลก',
 'ดี',
 'ห้อง',
 'ทาน',
 'หลัก',
 'อยู่',
 'ชั้น',
 'มี',
 'กาแฟ',
 'น้ำ',
 'ผึ้ง',
 'ซึ่ง',
 'ก็',
 'แค่',
 'เอา',
 'น้ำ',
 'ผึ้ง',
 'มา',
 'ราด',
 'แพง',
 'เวอร์',
 'อย่า',
 'สั่ง',
 'เลย',
 'ลาบไข่',
 'ต้ม',
 'ไข่',
 'มัน',
 'คาว',
 'อะ',
 'เลย',
 'ไม่',
 'ประทับใจ',
 'เท่า',
 'ไหร่',
 'ทอด',
 'มัน',
 'หัว',
 'ปลีก',
 'รอบ',
 'อร่อย',
 'ต้อง',
 'เบิ้ลพะแนง',
 'ห่อ',
 'ไข่',
 'อร่อย',
 'ดี',
 'เห้ย',
 'แต่',
 'ราคา',
 'บาท',
 'มัน',
 'เกิน',
 'ไป',
 'นะ',
 'รับ',
 'ไม่',
 'ไหวว',
 'เลิก',
 'กิน',
 'แล้ว',
 'มี',
 'ขนม',
 'หวาน',
 'ให้',
 'กิน',
 'ฟรี',
 'เล็กน้อย',
 'ขนม',
 'ไทย',
 'คง',
 'ไม่',
 'ไป',
 'ซ้ำ',
 'แพง',
 'เกิน']

In [25]:
%%time
debug['review_nospace'] = debug['review'].str.replace(' ', '')
debug_res4 = debug['review_nospace'].apply(lambda x: pythainlp.word_tokenize(x,
                                                                             engine='new_mm',
                                                                             keep_whitespace=False,
                                                                             custom_dict=debug_trie))

CPU times: user 14.8 ms, sys: 17 µs, total: 14.9 ms
Wall time: 31.7 ms


In [26]:
debug_res4[0]

['ร้านอาหาร',
 'ใหญ่มาก',
 'กกกกกกเลี้ยวเข้ามาเจอ',
 'ห้องน้ำ',
 'ก่อนเลยเออแปลกดีห้องทานหลักอยู่ชั้นมี',
 'กาแฟน้ำผึ้ง',
 'ซึ่งก็แค่เอา',
 'น้ำผึ้ง',
 'มาราดแพงเวอร์อย่า',
 'สั่งเลย',
 'ลาบไข่',
 'ต้มไข่',
 'มันคาวอะเลยไม่ประทับใจเท่าไหร่',
 'ทอดมันหัวปลี',
 'กรอบอร่อย',
 'ต้องเบิ้ล',
 'พะแนงห่อไข่',
 'อร่อยดี',
 'เห้ยแต่ราคาบาทมันเกินไปนะรับไม่ไหววเลิกกินแล้วมี',
 'ขนมหวาน',
 'ให้กินฟรีเล็กน้อย',
 'ขนมไทย',
 'คงไม่ไปซ้ำแพงเกิน']

deepcut result seem very promising, so we need to make it parallelized

# multiprocess deepcut

In [28]:
import numpy as np

In [60]:
train['chunk'] = np.random.randint(1,200 + 1,train.shape[0])
train['chunk'] = train['chunk'].apply(lambda x : f"{x:03}")
train.head()

,review,chunk
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,050
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,085
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,137
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,037
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,193


In [63]:
!ls ../data/train_chunk/source/

'chunk=001'  'chunk=035'  'chunk=069'  'chunk=103'  'chunk=137'  'chunk=171'
'chunk=002'  'chunk=036'  'chunk=070'  'chunk=104'  'chunk=138'  'chunk=172'
'chunk=003'  'chunk=037'  'chunk=071'  'chunk=105'  'chunk=139'  'chunk=173'
'chunk=004'  'chunk=038'  'chunk=072'  'chunk=106'  'chunk=140'  'chunk=174'
'chunk=005'  'chunk=039'  'chunk=073'  'chunk=107'  'chunk=141'  'chunk=175'
'chunk=006'  'chunk=040'  'chunk=074'  'chunk=108'  'chunk=142'  'chunk=176'
'chunk=007'  'chunk=041'  'chunk=075'  'chunk=109'  'chunk=143'  'chunk=177'
'chunk=008'  'chunk=042'  'chunk=076'  'chunk=110'  'chunk=144'  'chunk=178'
'chunk=009'  'chunk=043'  'chunk=077'  'chunk=111'  'chunk=145'  'chunk=179'
'chunk=010'  'chunk=044'  'chunk=078'  'chunk=112'  'chunk=146'  'chunk=180'
'chunk=011'  'chunk=045'  'chunk=079'  'chunk=113'  'chunk=147'  'chunk=181'
'chunk=012'  'chunk=046'  'chunk=080'  'chunk=114'  'chunk=148'  'chunk=182'
'chunk=013'  'chunk=047'  'chunk=081'  'chunk=115'  'chunk=149'  'chunk=183'

In [62]:
train.to_parquet(path='../data/train_chunk/source', partition_cols='chunk')

In [71]:
del deepcut

In [1]:
import os
os.environ["OPENBLAS_MAIN_FREE"] = "1"
os.system(f"taskset -p 0xff {os.getpid()}")

0

In [8]:
import pandas as pd
import glob

In [9]:
num_chunk_row = pd.read_parquet('../data/train_chunk/source/')
num_chunk_row = num_chunk_row.groupby('chunk',as_index=False).count()
num_chunk_row['chunk'] = num_chunk_row['chunk'].apply(lambda x: f"{x:03}")
num_chunk_row.head()

,chunk,review
0,001,161
1,002,186
2,003,212
3,004,198
4,005,184


In [20]:
num_complete = list()
for file in glob.glob('../data/train_chunk/result/dc_topword_food_dict/*'):
    num_complete.append(file.split('/')[-1].replace('.parq',''))
chunk_list = [f"{x:03}" for x in range(1,201)]
to_process = set(chunk_list).difference(set(num_complete))
to_process = list(to_process)
to_process

[]

In [10]:
for file in glob.glob('../data/train_chunk/result/dc_topword_food_dict/*'):
    num_ = file.split('/')[-1].replace('.parq','')
    row_src = num_chunk_row.loc[num_chunk_row['chunk'] == num_, 'review'].values[0]
    df_ = pd.read_parquet(file)
    row_res = df_.shape[0]
    assert row_res == row_src, f'the data for {num_} is not correct row_src {row_src} | row_res {row_res}'

In [4]:
!ls ../data/train_chunk/result/dc_topword_food_dict/

001.parq  008.parq  015.parq  022.parq	029.parq  042.parq  052.parq  059.parq
002.parq  009.parq  016.parq  023.parq	030.parq  043.parq  053.parq  060.parq
003.parq  010.parq  017.parq  024.parq	031.parq  044.parq  054.parq  061.parq
004.parq  011.parq  018.parq  025.parq	032.parq  045.parq  055.parq
005.parq  012.parq  019.parq  026.parq	033.parq  049.parq  056.parq
006.parq  013.parq  020.parq  027.parq	037.parq  050.parq  057.parq
007.parq  014.parq  021.parq  028.parq	041.parq  051.parq  058.parq


In [19]:
from multiprocessing import Pool
import multiprocessing

def tokenize_process(num_chunk):
    
    import deepcut
    import pandas as pd
    import gc
    
    import os
    import multiprocessing
    pool_size = multiprocessing.cpu_count()
    os.environ["OPENBLAS_MAIN_FREE"] = "1"
    os.system('taskset -cp 0-%d %s' % (pool_size, os.getpid()))
    
    def tokenize(s):
        import deepcut
        return deepcut.tokenize(s, custom_dict=tokenize_dict)
    
    def clean_special_char(train):

        special_char = r"[\\\@\^\%\(\*\|\<\`\.\_\=\!\>\;\?\#\$\)\/\,\ๆ\'\~\:\+\-\"\฿]"
        space_char = r"[\n\&]"
        num = '[0-9]'
        num_th = '[๑๒๓๓๔๕๖๗๘๙]'
        eng_words = '[A-Za-z]'

        train['review'] = train['review'].str.replace(special_char, ' ')
        train['review'] = train['review'].str.replace(space_char, ' ')
        train['review'] = train['review'].str.replace(num, ' ', regex=True)
        train['review'] = train['review'].str.replace(num_th, ' ', regex=True)
        train['review'] = train['review'].str.replace(eng_words, ' ', regex=True)
        train['review'] = train['review'].str.lower()

        return train
    
    print(f'read file from ../data/train_chunk/source/chunk={num_chunk}')
    train = pd.read_parquet(f'../data/train_chunk/source/chunk={num_chunk}')
    print(f'train shape {train.shape} for ../data/train_chunk/source/chunk={num_chunk}')
    
    print(f'clean_char : {num_chunk}')
    train = clean_special_char(train)
    train['review_nospace'] = train['review'].str.replace(' ', '')
    
    print(f'tokenizing : {num_chunk}')
    train['tokenized'] = train['review_nospace'].apply(tokenize)
    
    print(f'write to ../data/train_chunk/result/dc_topword_food_dict/{num_chunk}.parq')
    train.to_parquet(f'../data/train_chunk/result/dc_topword_food_dict/{num_chunk}.parq')
    
    del train
    gc.collect()
    return True

if __name__ == '__main__':
    with Pool(9) as P:
#         chunk_list = [f"{x:03}" for x in range(62,201)]
        P.map(func = tokenize_process, iterable = to_process)

read file from ../data/train_chunk/source/chunk=035
read file from ../data/train_chunk/source/chunk=036
train shape (207, 1) for ../data/train_chunk/source/chunk=035
clean_char : 035
read file from ../data/train_chunk/source/chunk=038
train shape (203, 1) for ../data/train_chunk/source/chunk=036
clean_char : 036
train shape (203, 1) for ../data/train_chunk/source/chunk=038
clean_char : 038
read file from ../data/train_chunk/source/chunk=034
tokenizing : 035
tokenizing : 036
tokenizing : 038
train shape (198, 1) for ../data/train_chunk/source/chunk=034
clean_char : 034
read file from ../data/train_chunk/source/chunk=039
read file from ../data/train_chunk/source/chunk=048
train shape (204, 1) for ../data/train_chunk/source/chunk=039
clean_char : 039
tokenizing : 034
train shape (182, 1) for ../data/train_chunk/source/chunk=048
clean_char : 048
tokenizing : 039
tokenizing : 048
read file from ../data/train_chunk/source/chunk=047
read file from ../data/train_chunk/source/chunk=046
train sh

key takeaways here:
    1. paralellizing deepcut is hard...
    2. https://shahhj.wordpress.com/2013/10/27/numpy-and-blas-no-problemo/ (make numpy, padnas not strict to single core)
    3. https://stackoverflow.com/questions/23537716/importing-scipy-breaks-multiprocessing-support-in-python/23546547
    4. https://stackoverflow.com/questions/15639779/why-does-multiprocessing-use-only-a-single-core-after-i-import-numpy
    5. https://stackoverflow.com/questions/50306632/multiprocessing-not-achieving-full-cpu-usage-on-dual-processor-windows-machine

# consolidate result

In [1]:
import pandas as pd
df = pd.read_parquet('../data/train_chunk/result/dc_topword_food_dict/')
df.shape

(40000, 3)

# remove stopwords

In [2]:
from collections import Counter
def get_top_freq_word(series, n):
    all_word = [a for b in series.tolist() for a in b]
    Counter_ = Counter(all_word)
    most_occur = Counter_.most_common(n) 
    return most_occur

In [39]:
keep_words = ['อร่อย','รสชาติ','ราคา','หวาน','ชอบ','เนื้อ','บริการ','ใหญ่','บรรยากาศ',
              'แนะนำ','พนักงาน','แพง','นาน','อร่อยมาก','น้ำจิ้ม','เครื่องดื่ม','น้ำซุป','เข้มข้น',
              'กาแฟ','มาลอง','อร่อยดี','ธรรมดา','ปกติ','ขนมปัง','ญี่ปุ่น','ชาเขียว','แปลก','ใช้ได้','⭐',
              'บรรยากาศร้าน','ก๋วยเตี๋ยว','หน้าตา','ครอบครัว','ราคาถูก','ปรับปรุง','สถานที่','ไส้กรอก','กลับบ้าน',
             'เป็นร้านเล็ก','พอใช้ได้']
top_remove_words = [w[0] for w in get_top_freq_word(df['tokenized'], n=1000) if w[0] not in keep_words]

In [40]:
df['tok_remove_topwords'] = df['tokenized'].apply(lambda x : [w for w in x if w not in top_remove_words])

In [41]:
get_top_freq_word(df['tok_remove_topwords'], n=50)

[('ราคา', 19677),
 ('รสชาติ', 18713),
 ('ชอบ', 13584),
 ('อร่อย', 12611),
 ('หวาน', 8368),
 ('บริการ', 7980),
 ('เนื้อ', 6944),
 ('ใหญ่', 6921),
 ('แนะนำ', 6893),
 ('พนักงาน', 6750),
 ('บรรยากาศ', 5757),
 ('อร่อยมาก', 4389),
 ('น้ำจิ้ม', 4257),
 ('เครื่องดื่ม', 4033),
 ('น้ำซุป', 3954),
 ('เข้มข้น', 3806),
 ('กาแฟ', 3652),
 ('ธรรมดา', 3539),
 ('นาน', 3383),
 ('มาลอง', 3251),
 ('อร่อยดี', 3218),
 ('ปกติ', 3030),
 ('แพง', 2324),
 ('ขนมปัง', 2147),
 ('ญี่ปุ่น', 2147),
 ('ชาเขียว', 2083),
 ('แปลก', 2069),
 ('ใช้ได้', 2054),
 ('⭐', 2037),
 ('บรรยากาศร้าน', 2005),
 ('ก๋วยเตี๋ยว', 1989),
 ('หน้าตา', 1165),
 ('ครอบครัว', 1159),
 ('ราคาถูก', 747),
 ('ปรับปรุง', 746),
 ('สถานที่', 744),
 ('ไส้กรอก', 739),
 ('กลับบ้าน', 736),
 ('เป็นร้านเล็ก', 736),
 ('พอใช้ได้', 733),
 ('ซ่า', 448),
 ('เป็นต้น', 448),
 ('โบราณ', 446),
 ('ซีฟู้ด', 446),
 ('หอมมาก', 445),
 ('เทศกาล', 445),
 ('ไม่ควรพลาด', 444),
 ('อาคาร', 443),
 ('ตบท้าย', 443),
 ('ทานที่ร้าน', 441)]

In [42]:
df['tok_remove_topwords'].apply(lambda x:len(x)).describe()

count    40000.000000
mean        24.088325
std         23.829494
min          0.000000
25%         12.000000
50%         17.000000
75%         28.000000
max       1187.000000
Name: tok_remove_topwords, dtype: float64

In [43]:
df['tokenized'].apply(lambda x:len(x)).describe()

count    40000.000000
mean       111.989425
std        112.933507
min          1.000000
25%         53.000000
50%         77.000000
75%        127.000000
max       4477.000000
Name: tokenized, dtype: float64

In [87]:
# df.to_parquet('../data/train_chunk/result/dc_topword_food_dict_remove_single_words_1K')

bigrams

In [177]:
%%time
import gensim
from nltk.util import ngrams
import nltk

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# bigram
# input_ = train['remove_top100_stopwords'].tolist()
# bigram = gensim.models.Phrases(input_, min_count=10, threshold=10) # higher threshold fewer phrases.
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# data_words_bigrams = make_bigrams(input_)
# train['bigrams'] = data_words_bigrams

# bigram

train['bigrams'] = train['remove_top100_stopwords'].apply(lambda row: list(nltk.ngrams(row, 2)))
train['bigrams'] = train['bigrams'].apply(lambda row: ['_'.join(gram) for gram in row])

# trigram
# trigram = gensim.models.Phrases(bigram[data_words], threshold=10)
# trigram_mod = gensim.models.phrases.Phraser(trigram)
# data_words_trigrams = make_trigrams(data_words_bigrams)

CPU times: user 1.78 s, sys: 204 ms, total: 1.98 s
Wall time: 1.98 s


In [184]:
train['bigrams'].sample(1).values

array([list(['ง่าย_ใจกลาง', 'ใจกลาง_เมือง', 'เมือง_แหนม', 'แหนม_เนืองรสชาติ', 'เนืองรสชาติ_ซื้อ', 'ซื้อ_ฝาก', 'ฝาก_ปริมาณ', 'ปริมาณ_กะ', 'กะ_ลำบาก', 'ลำบาก_บาง', 'บาง_ควร', 'ควร_น้อย', 'น้อย_ควร', 'ควร_น้อย', 'น้อย_แนะนำ', 'แนะนำ_ถาม', 'ถาม_น้อง', 'น้อง_พนักงาน', 'พนักงาน_ที', 'ที_ปริมาณ', 'ปริมาณ_เหมาะ', 'เหมาะ_กี่', 'กี่_ทยอย', 'ทยอย_ไม่', 'ไม่_เหลือ'])],
      dtype=object)

In [185]:
get_top_freq_word(train['bigrams'], n=30)

[('อร่อย_ดี', 6293),
 ('ไม่_แพง', 4997),
 ('ราคา_ไม่', 4935),
 ('กำลัง_ดี', 3744),
 ('ดี_ไม่', 3647),
 ('บริการ_ดี', 3613),
 ('ของ_หวาน', 2946),
 ('อร่อย_ไม่', 2672),
 ('รสชาติ_อร่อย', 2646),
 ('ไม่_อร่อย', 2607),
 ('รสชาติ_ดี', 2500),
 ('ไม่_รู้', 2411),
 ('ไม่_หวาน', 2280),
 ('ผิด_หวัง', 2230),
 ('ไม่_ชอบ', 2223),
 ('บรรยากาศ_ดี', 2126),
 ('ไม่_เคย', 2125),
 ('รสชาติ_ไม่', 2014),
 ('ไม่_ใช่', 1948),
 ('พนักงาน_บริการ', 1798),
 ('กลับ_บ้าน', 1747),
 ('ไม่_นาน', 1725),
 ('ไม่_ผิด', 1704),
 ('คุ้ม_ค่า', 1691),
 ('ไม่_ไม่', 1561),
 ('ถ่าย_รูป', 1559),
 ('หมูก_รอบ', 1478),
 ('ไม่_ถูก', 1375),
 ('⭐_️', 1318),
 ('ริม_ถนน', 1268)]

In [9]:
def tokenize_word(df, stop_words = None, trie = None):
    
    # tokenized
    if trie is None:
        df['word_tokenized'] = df['review'].apply(lambda x: word_tokenize(x,engine='attacut',keep_whitespace=False))
        df['len_review'] = df['word_tokenized'].apply(lambda x: len(x))
        print('finish tokenize word')
    else:
        df['word_tokenized'] = df['review'].apply(lambda x: word_tokenize(x,engine='attacut',keep_whitespace=False,custom_dict=trie))
        df['len_review'] = df['word_tokenized'].apply(lambda x: len(x))
        print('finish tokenize word with custom_dict')

    # remove duplicated word
    df['word_tokenized_uniq'] = df['word_tokenized'].apply(lambda x: list(set(x)))
    df['len_review_uniq'] = df['word_tokenized_uniq'].apply(lambda x: len(x))
    print('finish remove duplicated word')

    # remove stopword
    thai_stopword_ls = list(pythainlp.corpus.thai_stopwords())
    df['word_tokenized_uniq_cls_revstopw'] = df['word_tokenized_uniq'].apply(lambda x : [w for w in x if w not in thai_stopword_ls])
    df['len_review_uniq_cls_revstopw'] = df['word_tokenized_uniq_cls_revstopw'].apply(lambda x: len(x))
    print('finish remove basic stopwords')
    
    if stop_words is not None:
        df['word_tokenized_uniq_cls_revstopw'] = df['word_tokenized_uniq_cls_revstopw'].apply(lambda x : [w for w in x if w not in stop_words])
        df['len_review_uniq_cls_revstopw'] = df['word_tokenized_uniq_cls_revstopw'].apply(lambda x: len(x))
        print('finish remove additional stopwords')

    # describe stat
    display(df[['len_review','len_review_uniq','len_review_uniq_cls_revstopw']].describe().transpose())
    
    return df

In [10]:
train_clean = tokenize_word(train_clean, trie=trie)

finish tokenize word with custom_dict
finish remove duplicated word
finish remove basic stopwords


,count,mean,std,min,25%,50%,75%,max
len_review,10000.0,141.8193,122.378546,10.0,67.0,104.0,166.25,1646.0
len_review_uniq,10000.0,89.4904,51.873278,4.0,53.0,75.0,108.00,511.0
len_review_uniq_cls_revstopw,10000.0,51.0161,33.320343,3.0,29.0,41.0,61.00,352.0


create bigrams/trigrams/quardgrams

In [11]:
from collections import Counter
def get_top_freq_word(series, n):
    all_word = [a for b in series.tolist() for a in b]
    Counter_ = Counter(all_word)
    most_occur = Counter_.most_common(n) 
    return most_occur

from nltk.util import ngrams
import nltk
train['bigrams'] = train['word_tokenized'].apply(lambda row: list(nltk.ngrams(row, 2)))

def create_n_grams(train):
    
    train['bigrams'] = train['word_tokenized'].apply(lambda row: list(nltk.ngrams(row, 2)))
    train['trigrams'] = train['word_tokenized'].apply(lambda row: list(nltk.ngrams(row, 3)))
    train['quardgrams'] = train['word_tokenized'].apply(lambda row: list(nltk.ngrams(row, 4)))
    
    train['bigrams'] = train['bigrams'].apply(lambda row: ['_'.join(gram) for gram in row])
    train['trigrams'] = train['trigrams'].apply(lambda row: ['_'.join(gram) for gram in row])
    train['quardgrams'] = train['quardgrams'].apply(lambda row: ['_'.join(gram) for gram in row])
    train['bi_tri_grams'] = train['bigrams'] + train['trigrams']
    train['all_grams'] = train['word_tokenized'] + train['bigrams'] + train['trigrams'] + train['quardgrams']
    
    return train

In [12]:
train_clean = create_n_grams(train_clean)

In [13]:
train_clean.head(1)

,review,word_tokenized,len_review,word_tokenized_uniq,len_review_uniq,word_tokenized_uniq_cls_revstopw,len_review_uniq_cls_revstopw,bigrams,trigrams,quardgrams,bi_tri_grams,all_grams
0,ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำ...,"[ร้าน, อาหาร, ใหญ่, มาก, กกกกกก, เลี้ยว, เข้า,...",89,"[นะ, ร้าน, ต้อง, มาก, ซึ่ง, ผึ้ง, พะแนง, เล็กน...",73,"[ร้าน, ผึ้ง, พะแนง, ต้ม, น้ำ, ไทย, เจอ, แปลก, ...",48,"[ร้าน_อาหาร, อาหาร_ใหญ่, ใหญ่_มาก, มาก_กกกกกก,...","[ร้าน_อาหาร_ใหญ่, อาหาร_ใหญ่_มาก, ใหญ่_มาก_กกก...","[ร้าน_อาหาร_ใหญ่_มาก, อาหาร_ใหญ่_มาก_กกกกกก, ใ...","[ร้าน_อาหาร, อาหาร_ใหญ่, ใหญ่_มาก, มาก_กกกกกก,...","[ร้าน, อาหาร, ใหญ่, มาก, กกกกกก, เลี้ยว, เข้า,..."


In [15]:
top_words = get_top_freq_word(train_clean['bigrams'], 50)
top_words_to_clean = [c[0] for c in top_words if c[0] not in ['อร่อย_มาก','ไม่_แพง','ส่วน_ตัว','ใช้_ได้','อร่อย_ดี']]
train_clean['bi_tri_grams'] = train_clean['bi_tri_grams'].apply(lambda row: [c for c in row if c not in top_words_to_clean])

In [16]:
top_words = get_top_freq_word(train_clean['trigrams'], 50)
top_words_to_clean = [c[0] for c in top_words if c[0] not in ['ราคา_ไม่_แพง','ใช้_ได้_เลย','ไม่_ผิด_หวัง','อร่อย_ใช้_ได้']]
train_clean['bi_tri_grams'] = train_clean['bi_tri_grams'].apply(lambda row: [c for c in row if c not in top_words_to_clean])

In [17]:
# output
data_words = train['bi_tri_grams'].tolist()

In [18]:
train['bi_tri_grams'].apply(len).describe()

count    10000.000000
mean       271.470500
std        237.836188
min         14.000000
25%        126.000000
50%        198.000000
75%        319.000000
max       3198.000000
Name: bi_tri_grams, dtype: float64

# feature engineer

In [21]:
import gensim
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

 bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=10, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# add bigram trigram
data_words_bigrams = make_bigrams(data_words)
data_words_trigrams = make_trigrams(data_words_bigrams)

In [35]:
thai_stopword_ls = list(pythainlp.corpus.thai_stopwords())
train['tokenized'] = train['word_tokenized'].apply(lambda x : [w for w in x if w not in thai_stopword_ls])

In [36]:
data_words = train['tokenized'].tolist()

In [37]:
%%time
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=10, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# add bigram trigram
data_words_bigrams = make_bigrams(data_words)
data_words_trigrams = make_trigrams(data_words_bigrams)

16.8 s ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
train['bigrams2'] = data_words_bigrams

In [47]:
train['bigrams2'][2]

['ปอเปี๊ยะ_สด',
 'สด_ทุก_ทุก_วัน',
 'นี้_รู้สึก_รู้สึก_ว่า',
 'ว่า_หากิน',
 'หากิน_ยาก',
 'ยาก_ร้าน',
 'ที่_ขาย',
 'ขาย_ปอเปี๊ยะ',
 'ปอเปี๊ยะ_สด',
 'สด_อย่าง',
 'อย่าง_เดียว',
 'เดียว_ส่วน',
 'ส่วน_มาก_มาก_ที่',
 'ที่_เจอ',
 'เจอ_คือ',
 'คือ_ขาย',
 'ขาย_ตาม',
 'ตาม_ภัตตาคาร',
 'ภัตตาคาร_ซึ่ง',
 'ซึ่ง_มัก',
 'มัก_จะ_จะ_ไม่',
 'ค่อย_อร่อย',
 'อร่อย_ร้าน',
 'นี้_ที่',
 'ที่_สะพานเหลือง',
 'สะพานเหลือง_ราคา',
 'ราคา_ย่อมเยา',
 'ย่อมเยา_เพียง',
 'เพียง_แค่_แค่_บาท',
 'บาท_จึง',
 'จึง_ซื้อ',
 'ซื้อ_กิน',
 'กิน_ทุก_ทุก_ครั้ง',
 'ครั้ง_ที่_ที่_มา',
 'มา_รสชาติ_รสชาติ_ดี',
 'ดี_พอสมควร',
 'พอสมควร_เลย',
 'เลย_โดย_โดย_เฉพาะ',
 'เฉพาะ_เต้าหู้',
 'เต้าหู้_กับ',
 'กับ_กุนเชียง',
 'กุนเชียง_ที่',
 'ที่_ใส่_ใส่_มา',
 'มา_ให้_ให้_อย่าง',
 'อย่าง_จุใจ',
 'จุใจ_ราด',
 'ราด_น้ำ',
 'จิ้ม_ที่',
 'ที่_โอเค',
 'โอเค_ไม่_ไม่_ถึง',
 'ถึง_กับ',
 'กับ_สุด',
 'สุด_ยอด',
 'ยอด_เสิร์ฟ',
 'เสิร์ฟ_พรัอม',
 'พรัอม_เครื่อง',
 'เครื่อง_เคียง',
 'เคียง_ต้น',
 'ต้น_หอม',
 'หอม_กับ',
 'กับ_พริก',
 'พริก_เขียว',
 'เขียว_มา'

In [22]:
import gensim.corpora as corpora

def get_lda_input(data_words):
    
    # Create Dictionary
    id2word = corpora.Dictionary(data_words)
        
    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in data_words]
    
    return id2word, corpus

In [29]:
from gensim.models import CoherenceModel

def run_simple_model(list_of_data_words):
    
    for data_words in list_of_data_words:

        id2word, corpus = get_lda_input(data_words)

        # Build LDA model
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=10, 
                                               random_state=100,
                                               chunksize=100,
                                               passes=10,
                                               per_word_topics=True)

        # Compute Coherence Score
        coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()

        print('\nCoherence Score: ', coherence_lda)

In [30]:
run_simple_model([data_words])


Coherence Score:  0.4845599722051414


Tuning topics

In [ ]:
def compute_coherence_values(texts, corpus, id2word, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm

def grid_search(data_words, ofile_nm):
    
    id2word, corpus = get_lda_input(data_words)
    print('finish get_lda_input')

    grid = {}
    grid['Validation_Set'] = {}

    # Topics range
    min_topics = 2
    max_topics = 11
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)

    # Alpha parameter
    alpha = list(np.arange(0.01, 1, 0.3))
    alpha.append('symmetric')
    alpha.append('asymmetric')

    # Beta parameter
    beta = list(np.arange(0.01, 1, 0.3))
    beta.append('symmetric')

    # Validation sets
    num_of_docs = len(corpus)
    corpus_sets = [corpus]
    corpus_title = ['100% Corpus']
    model_results = {'Validation_Set': [],
                     'Topics': [],
                     'Alpha': [],
                     'Beta': [],
                     'Coherence': []
                    }
    
    print('finish initial storage list')
    
    # Can take a long time to run
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    try:
                        cv = compute_coherence_values(corpus=corpus_sets[i], id2word=id2word, texts=data_words,
                                                      k=k, a=a, b=b)
                        # Save the model results
                        model_results['Validation_Set'].append(corpus_title[i])
                        model_results['Topics'].append(k)
                        model_results['Alpha'].append(a)
                        model_results['Beta'].append(b)
                        model_results['Coherence'].append(cv)

                        print(f'finish loop k : {k}, a : {a}, b : {b}, score : {cv:.4f}')
                    except:
                        print(f'error in loop k : {k}, a : {a}, b : {b}, score : {cv:.4f}')

    pd.DataFrame(model_results).to_csv(ofile_nm, index=False)

In [ ]:
grid_search(data_words_trigrams, ofile_nm='tuning_data_words_trigram_5K_docs.csv')

In [ ]:
_ = pd.read_csv('tuning_data_words_trigram_5K_docs.csv')
_.sort_values('Coherence',ascending=False).head()

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import seaborn as sns
sns.lineplot(x='Alpha', y='Coherence', data=_);

In [ ]:
sns.lineplot(x='Topics', y='Coherence', data=_);

In [ ]:
sns.lineplot(x='Beta', y='Coherence', data=_);

fit optimized model

In [26]:
import mlflow
import warnings
warnings.filterwarnings(action='ignore',category=FutureWarning)
warnings.filterwarnings(action='ignore',category=DeprecationWarning)

try:
    mlflow.create_experiment(name='lda_development')
except:
    expr = mlflow.get_experiment_by_name('lda_development')

In [25]:
import gensim
import joblib

In [1]:
id2word, corpus = get_lda_input(data_words)
k = 16

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=k,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
joblib.dump(lda_model, 'test_10Kdocs_30topics_model.pkl')

Visualization

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

In [ ]:
%%time
lda_10K = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=id2word, n_jobs=12)

In [ ]:
pyLDAvis.display(lda_10K)

# Fit [BOW]

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [ ]:
processed_docs = train['word_tokenized_uniq_cls_revstopw'].copy()

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[500]

In [ ]:
ind = 500
bow_doc = bow_corpus[ind]
for i in range(len(bow_doc)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc[i][0], 
                                               dictionary[bow_doc[i][0]], 
bow_doc[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary,
                                       passes=100, workers=2, random_state = 0)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print('\n')
    print('Topic: {} \n@Words: {}'.format(idx, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=30, id2word=dictionary,
                                             passes=20, workers=4, random_state=0)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('\n')
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
for ind in range(0,500,75):
    print("*"*30,ind,"*"*30)
    print([train['review'][ind]])
    for index, score in sorted(lda_model[bow_corpus[ind]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 20)))

In [ ]:
for ind in range(0,500,75):
    print("*"*30,ind,"*"*30)
    print([train['review'][ind]])
    for index, score in sorted(lda_model_tfidf[bow_corpus[ind]], key=lambda tup: -1*tup[1]):
        print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 20)))